In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-pdf/Joseph Babcock Raghav Bali - Generative AI with Python and TensorFlow 2_ Create images text and music with VAEs GANs LSTMs Transformer models-Packt Publishing (2021).pdf
/kaggle/input/ai-pdf/Ben Auffarth - Generative AI with LangChain_ Build large language model (LLM) apps with Python ChatGPT and other LLMs-Packt (2023).pdf
/kaggle/input/ai-pdf/Chris Fregly Antje Barth Shelbee Eigenbrode - Generative AI on AWS_ Building Context-Aware Multimodal Reasoning Applications-OReilly Media Inc. (2023).pdf
/kaggle/input/ai-pdf/Python for Programmers with Big Data and Artificial Intelligence Case Studies by Paul J. Deitel Harvey Deitel (z-lib.org).pdf


In [96]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_HF = user_secrets.get_secret("Huggingface")
secret_value_OA = user_secrets.get_secret("OPEN_API_KEY")
secret_value_PC = user_secrets.get_secret("PINECONE")
secret_value_Wbd = user_secrets.get_secret("wandb login")
secret_value_A_KEY = user_secrets.get_secret("ASTRA_API_KEY")
secret_value_A_DB = user_secrets.get_secret("ASTRA_DB_ID")
secret_value_P_ENV = user_secrets.get_secret("PINECONE_ENV")

In [4]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [6]:
import os




In [12]:
#!pip install PyPDFLoader

ERROR: Could not find a version that satisfies the requirement PyPDFLoader (from versions: none)
ERROR: No matching distribution found for PyPDFLoader


In [168]:
from langchain.document_loaders import PyPDFLoader
loaders=[
    #PyPDFLoader('/kaggle/input/ai-pdf/Ben Auffarth - Generative AI with LangChain_ Build large language model (LLM) apps with Python ChatGPT and other LLMs-Packt (2023).pdf'),
    PyPDFLoader('/kaggle/input/ai-pdf/Chris Fregly Antje Barth Shelbee Eigenbrode - Generative AI on AWS_ Building Context-Aware Multimodal Reasoning Applications-OReilly Media Inc. (2023).pdf'),
    #PyPDFLoader('/kaggle/input/ai-pdf/Joseph Babcock Raghav Bali - Generative AI with Python and TensorFlow 2_ Create images text and music with VAEs GANs LSTMs Transformer models-Packt Publishing (2021).pdf'),
    #PyPDFLoader('/kaggle/input/ai-pdf/Python for Programmers with Big Data and Artificial Intelligence Case Studies by Paul J. Deitel Harvey Deitel (z-lib.org).pdf')
    
]

docs=[]
for loader in loaders:
    docs.extend(loader.load())

In [169]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=10000,
    chunk_overlap=1000
)

splits=text_splitter.split_documents(docs)

In [170]:
#pip install openai

In [171]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings(openai_api_key=secret_value_OA)

In [172]:
"""from langchain.vectorstores import Chroma
vectordb=Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory='RAG_BOT'
)
"""

"from langchain.vectorstores import Chroma\nvectordb=Chroma.from_documents(\n    documents=splits,\n    embedding=embeddings,\n    persist_directory='RAG_BOT'\n)\n"

In [173]:
#pip install FAISS-cpu


In [174]:
#pip install tiktoken

!pip install -qU \
 pinecone-client==3.0.0

In [175]:
from langchain.vectorstores import FAISS

vectordb=FAISS.from_documents(splits,embeddings)
vectordb.save_local('RAG_BOT_')

In [176]:
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(openai_api_key=secret_value_OA,model_name="gpt-3.5-turbo",temperature=0)

In [177]:
from langchain.prompts import PromptTemplate
template="""Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question:{question}
Helpful Answer:"""
QA_CHAIN_PROMPT=PromptTemplate(input_variables=["context","Question"],template=template)

#run chain
from langchain.chains import RetrievalQA
qa_chain=RetrievalQA.from_chain_type(llm,
                                     retriever=vectordb.as_retriever(),
                                     return_source_documents=True,
                                     chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})#search_type="mmr"

In [178]:
from langchain.memory import ConversationBufferMemory
memory=ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [180]:
"""from langchain.chains.conversation.memory import ConversationBufferWindowMemory

conversation = ConversationChain(
	llm=llm,
	memory=ConversationBufferWindowMemory(k=1)
)
"""

'from langchain.chains.conversation.memory import ConversationBufferWindowMemory\n\nconversation = ConversationChain(\n\tllm=llm,\n\tmemory=ConversationBufferWindowMemory(k=1)\n)\n'

In [181]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa=ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory)#ConversationBufferWindowMemory(k=1,memory_key="chat_history",return_messages=True)

In [182]:
q1="what is RAG?"
result=qa({"question":q1})
result["answer"]

'RAG stands for Retrieval-Augmented Generation. It is a framework that provides Language Model (LLM) powered applications access to external data sources and applications that were not seen during training. RAG allows the model to have access to additional data beyond what it learned during pretraining and fine-tuning, improving model completion relevance and mitigating challenges like knowledge limitations and hallucinations.'

In [183]:
q2="explain Transformers?"
result=qa({"question":q2})
result["answer"]

'Transformers are a type of neural network architecture that was introduced in 2017 and are at the core of many modern language models. They are used in various language tasks, such as generating completions for input prompts, gaining contextual understanding from training data, and creating personalized content. Transformers consist of components like embeddings, self-attention layers, encoders, decoders, and softmax outputs, and they are essential for tasks like text classification and generative tasks. Transformers have different variants, including encoder-only, decoder-only, and encoder-decoder models, each trained with different objectives to address various generative tasks.'

In [184]:
q3="explain its architecture "
result=qa({"question":q3})
result["answer"]

'The architecture of Transformers consists of components such as input token context window, embeddings, encoder, self-attention layers, decoder, and softmax output. The model generates new tokens in a loop until a stop condition is reached, often signaled by an end-of-sequence (EOS) token. The Transformer architecture is a key component in generative models, particularly in language-related tasks.'

In [185]:
q4="explain chain of thoughts?"
result=qa({"question":q4})
result["answer"]

'A chain of thoughts is a reasoning strategy that involves breaking down a problem or task into sequential steps or thought processes. Each step in the chain builds upon the previous one, leading to a logical progression towards a solution or understanding of the issue at hand. This method helps guide the thought process systematically and can be used to prompt large language models to elicit reasoning and generate accurate responses.'

In [186]:
q5="give example of code"
result=qa({"question":q5})
result["answer"]

'Sure, here is an example of code for implementing the Transformer architecture using the Hugging Face Transformers Python library:\n\n```python\nimport torch\nfrom transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments\n\n# Load the pre-trained model and tokenizer\nmodel = AutoModelForCausalLM.from_pretrained("distilgpt2")\ntokenizer = AutoTokenizer.from_pretrained("distilgpt2")\n\n# Define training arguments\ntraining_args = TrainingArguments(\n    output_dir="./results",  # output directory\n    num_train_epochs=3,  # number of training epochs\n    per_device_train_batch_size=2,  # batch size per device during training\n    save_steps=10_000,  # save checkpoint every 10,000 steps\n    save_total_limit=2,  # limit the total amount of saved checkpoints\n)\n\n# Define trainer\ntrainer = Trainer(\n    model=model,  # the instantiated 🤗 Transformers model to be trained\n    args=training_args,  # training arguments\n    train_dataset=dataset,  # training data

from pinecone import Pinecone as PineconeClient

from langchain.vectorstores import Chroma, Pinecone
from pinecone import Pinecone
Pinecone(
    api_key=secret_value_PC,  # find at app.pinecone.io
    environment=secret_value_P_ENV  # next to api key in console
 )
index_name = "ragbot" # put in the name of your pinecone index here


index = Pinecone.Index(index_name)

docsearch = Pinecone.from_texts([t.page_content for t in splits], embeddings, index_name=index_name)

